# Run Prediction

### 1. Take in input video (Configure here)

In [9]:
#key in input filename here
input_vid_fn = 'multi2.mp4'
#set to true if videos are taken from mobile phone
mobile = True
#set to true if using custom metrics to evaluate model (Default is true as it uses get_f1 in utils)
custom_metric = True
#set name of model to be used (Set this only if a custom model was created, default model is seq30_128dense_lost0.58_f1-0.7_100x700.h5)
model_name = "seq30_128dense_lost0.58_f1-0.7_100x700.h5"
#set this to be same with the architecture of custom model used
IMG_HEIGHT, IMG_WIDTH = 700,100 #this is the size of images taken in for "seq30_128dense_lost0.58_f1-0.7_100x700.h5"



In [3]:
from pathlib import Path
import os,sys, tensorflow as tf, numpy as np
from os.path import exists, join, basename
import subprocess
BASE_DIR = os.getcwd()
called_dir = BASE_DIR
while os.path.basename(BASE_DIR) != "fyp_team4c":
    path = Path(BASE_DIR)
    BASE_DIR = str(path.parent)
    if BASE_DIR == '/':
        print("Please call this script in the fyp_team4c directory")
        break

PREDICTION_DIR = os.path.join(BASE_DIR, 'prediction')
sys.path.append(BASE_DIR)
from utils import *
from prepare_img_data import load_predict_data_vid, normalize_img_data
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

### 2. Run DeepSORT on input video

In [4]:
output_dir = os.path.join(PREDICTION_DIR, 'output')
input_vid_p = os.path.join(PREDICTION_DIR, f'input_vid/{input_vid_fn}')

run_deepsort(output_dir, input_vid_p, mobile, called_dir, BASE_DIR)


/home/student/jiawen/project/fyp_team4c/deep_sort_pytorch
/home/student/jiawen/project/fyp_team4c/prediction/output/final_vid
/home/student/jiawen/project/fyp_team4c/prediction/output/bbox_output
/home/student/jiawen/project/fyp_team4c/prediction/input_vid


### 3. Extract Frames to readable input for model.

In [5]:
#Extract frames to frames folder
frames_folder = os.path.join(output_dir, 'frames')
if not os.path.exists(frames_folder):
  # get_ipython().system('cd OUTPUT_DIR && mkdir frames')
  os.makedirs(frames_folder)
print('Extracting: ',input_vid_fn)
extract_frames(input_vid_p, input_vid_fn, frames_folder)

Extracting:  multi2.mp4
Extracting  /home/student/jiawen/project/fyp_team4c/prediction/input_vid/multi2.mp4 to  /home/student/jiawen/project/fyp_team4c/prediction/output/frames


In [6]:
#Extract bbox of the frames
bbox_dir = os.path.join(output_dir, 'bbox_output')
print("Extracting ", input_vid_fn)
pickle_path = f'{bbox_dir}/{input_vid_fn[:-4]}_bbox.pkl'
extract_bbox(input_vid_fn[:-4], pickle_path, frames_folder)


Extracting  multi2.mp4


### 4. Read in the trained model

In [10]:
model_fp = os.path.join(PREDICTION_DIR, os.path.join("model", model_name))

if os.path.exists(model_fp):
    if custom_metric:
        model = tf.keras.models.load_model(
        model_fp, custom_objects={"get_f1":get_f1}, compile=True
    )
    else:
        model = tf.keras.models.load_model(
        model_fp, custom_objects=None, compile=True
    )
else:
    print("Missing model for prediction!")
    raise Error("Missing model")

### 5. Make prediction

In [14]:
vid_frames_folder = os.path.join(frames_folder, input_vid_fn[:-4])
my_x, names, images= load_predict_data_vid(vid_frames_folder, input_vid_fn[:-4])
my_x = normalize_img_data(my_x, IMG_HEIGHT, IMG_WIDTH)
my_x = np.array(my_x)
print(my_x.shape)
res = []
for i in range(len(my_x)):
    val = model.predict(np.array([my_x[i,]]))[0]
    # y_pred.append(0 if val <= 0.5 else 1)
    print(f"Predicted prob: {val}\tname: {names[i]}")
    res.append({
    "ID": names[i],
    "filepath": images[i],
    "mood": ("Depressed" if val < 0.5 else "Healthy"),
    "conf": str(val[0])
    })
print("Results")
print(res)

301 85
302 72
(2, 30, 700, 100, 1)
Predicted prob: [0.30185026]	name: multi2-id_01
Predicted prob: [0.48002073]	name: multi2-id_02
Results
[{'ID': 'multi2-id_01', 'filepath': '/home/student/jiawen/project/fyp_team4c/prediction/output/frames/multi2/id_01/frame074_id01.jpg', 'mood': 'Sad', 'conf': '0.30185026'}, {'ID': 'multi2-id_02', 'filepath': '/home/student/jiawen/project/fyp_team4c/prediction/output/frames/multi2/id_02/frame060_id02.jpg', 'mood': 'Sad', 'conf': '0.48002073'}]
